### Instance generator

In [8]:
using Random

function generate_random_symmetric_tsp(n::Int, max_cost::Int=100)
    C = zeros(Int, n, n)
    for i in 1:n-1
        for j in i+1:n
            C[i, j] = rand(1:max_cost)
            C[j, i] = C[i, j]
        end
    end
    return C
end

# Generate 5 random TSP instances with n=10
tsp_instances = [generate_random_symmetric_tsp(10) for _ in 1:5]

# Print each instance
for (idx, mat) in enumerate(tsp_instances)
    println("Instance $idx:")
    println(mat)
    println()
end

Instance 1:
[0 81 97 86 76 16 1 42 48 1; 81 0 53 12 93 16 16 17 50 88; 97 53 0 98 24 51 36 92 82 83; 86 12 98 0 28 34 93 16 73 54; 76 93 24 28 0 67 99 15 81 88; 16 16 51 34 67 0 30 5 23 20; 1 16 36 93 99 30 0 68 75 83; 42 17 92 16 15 5 68 0 39 49; 48 50 82 73 81 23 75 39 0 81; 1 88 83 54 88 20 83 49 81 0]

Instance 2:
[0 84 61 92 27 17 19 4 96 16; 84 0 68 59 94 31 35 76 77 40; 61 68 0 56 49 99 2 67 70 38; 92 59 56 0 93 21 14 52 47 18; 27 94 49 93 0 23 28 60 65 89; 17 31 99 21 23 0 91 27 95 92; 19 35 2 14 28 91 0 7 61 76; 4 76 67 52 60 27 7 0 93 68; 96 77 70 47 65 95 61 93 0 47; 16 40 38 18 89 92 76 68 47 0]

Instance 3:
[0 87 67 25 82 12 96 56 68 93; 87 0 6 6 66 79 85 22 89 80; 67 6 0 25 72 83 3 81 19 46; 25 6 25 0 33 52 69 96 63 2; 82 66 72 33 0 26 34 74 53 4; 12 79 83 52 26 0 46 43 86 44; 96 85 3 69 34 46 0 59 1 49; 56 22 81 96 74 43 59 0 81 75; 68 89 19 63 53 86 1 81 0 36; 93 80 46 2 4 44 49 75 36 0]

Instance 4:
[0 6 25 1 76 75 85 61 88 52; 6 0 50 54 59 24 48 43 65 86; 25 50 0 62 5

### (a) MTZ formulation

In [3]:
using JuMP, Gurobi, Statistics

In [4]:
function solve_mtz_tsp(C::Matrix{Int})
    n = size(C, 1)
    model = Model(Gurobi.Optimizer)
    
    set_silent(model)

    # Binary decision variables x[i,j]
    @variable(model, x[1:n, 1:n], Bin)
    # MTZ auxiliary variables with bounds 2 <= u[i] <= n for i=2..n
    @variable(model, 2 <= u[2:n] <= n)

    # Objective: minimize total cost
    @objective(model, Min, sum(C[i,j] * x[i,j] for i in 1:n, j in 1:n))

    # Degree constraints: one in, one out per city
    @constraint(model, [i=1:n], sum(x[i, j] for j in 1:n) == 1)
    @constraint(model, [j=1:n], sum(x[i, j] for i in 1:n) == 1)

    # MTZ subtour elimination
    @constraint(model, [i=2:n, j=2:n; i != j], u[i] - u[j] + n * x[i, j] <= n - 1)

    # No self-loops
    @constraint(model, [i=1:n], x[i, i] == 0)

    # Solve and return time & tour length
    elapsed = @elapsed optimize!(model)
    tour_length = objective_value(model)
    return elapsed, tour_length
end

solve_mtz_tsp (generic function with 1 method)

### (b) DFJ formulation + lazy constraints

In [26]:
using MathOptInterface

#### Helper Method

In [5]:
# Helper to find subtours in a solution matrix
function find_subtours(sol::Matrix{Int})

    n = size(sol,1)
    visited = falses(n)
    subtours = Vector{Vector{Int}}()

    for i in 1:n
        if !visited[i]

            queue = [i]
            comp = Int[]
            visited[i] = true

            while !isempty(queue)
                u = popfirst!(queue)
                push!(comp,u)

                for v in 1:n
                    if sol[u,v] == 1 && !visited[v]
                        visited[v] = true
                        push!(queue,v)
                    end
                end

            end

            push!(subtours, comp)

        end
    end
    
    return subtours

end

find_subtours (generic function with 1 method)

#### Formulation

In [45]:
function solve_dfj_tsp(C::Matrix{Int})
    n = size(C,1)
    model = Model(Gurobi.Optimizer)

    set_optimizer_attribute(model, "LazyConstraints", 1)
    set_silent(model)

    @variable(model, x[1:n,1:n], Bin)
    @constraint(model, [i=1:n], sum(x[i,j] for j in 1:n) == 1)
    @constraint(model, [j=1:n], sum(x[i,j] for i in 1:n) == 1)
    @constraint(model, [i=1:n], x[i,i] == 0)
    @objective(model, Min, sum(C[i,j]*x[i,j] for i in 1:n, j in 1:n))

    function dfj_callback(cb_data)
        # Extract current solution
        sol = [ callback_value(cb_data, x[i,j]) > 0.5 ? 1 : 0 for i in 1:n, j in 1:n ]
        # Add lazy constraints for any subtour smaller than full tour
        for tour in find_subtours(sol)
            if length(tour) < n
                # f = sum(x[i,j] for i in tour, j in tour)
                # s = MathOptInterface.LessThan(length(tour)-1)
                con = @build_constraint(sum(x[i,j] for i in tour, j in tour) <= length(tour)-1)
                MathOptInterface.submit(model, MOI.LazyConstraint(cb_data), con)
            end
        end
    end
    # Register callback
    MathOptInterface.set(model, MathOptInterface.LazyConstraintCallback(), dfj_callback)
    elapsed = @elapsed optimize!(model)
    return elapsed, objective_value(model)

end

solve_dfj_tsp (generic function with 1 method)

### (c) 2-opt heuristic

In [9]:
function tour_length(C::Matrix{Int}, tour::Vector{Int})
    n = length(tour)
    total = 0
    for i in 1:n-1
        total += C[tour[i], tour[i+1]]
    end
    total += C[tour[n], tour[1]]
    return total
end

function solve_2opt_tsp(C::Matrix{Int})
    n = size(C,1)

    # randomize
    tour = shuffle(collect(1:n))
    best_cost = tour_length(C, tour)
    improved = true

    elapsed = @elapsed begin
        while improved
            improved = false
            for i in 1:n-2
                for j in i+2:n
                    if !(i == 1 && j == n)
                        new_tour = vcat(tour[1:i], reverse(tour[i+1:j]), tour[j+1:end])
                        new_cost = tour_length(C, new_tour)
                        if new_cost < best_cost
                            tour = new_tour
                            best_cost = new_cost
                            improved = true
                            break
                        end
                    end
                end
                if improved
                    break
                end
            end
        end
    end
    return elapsed, best_cost
end

solve_2opt_tsp (generic function with 1 method)

### (d) Concorde.jl package

In [11]:
using Concorde

In [49]:
function solve_concorde_tsp(C::Matrix{Int})
    # Measure time to solve and capture tour
    tour = Vector{Int}()
    elapsed = @elapsed begin
        tour, length = Concorde.solve_tsp(C)
    end
    return elapsed, length
end

solve_concorde_tsp (generic function with 1 method)

### (e) LKH.jl package

In [12]:
using LKH

In [54]:
function solve_lkh_tsp(C::Matrix{Int})
    tour = Vector{Int}()
    elapsed = @elapsed begin
        tour, length = LKH.solve_tsp(C)
    end
    return elapsed, length
end

solve_lkh_tsp (generic function with 1 method)

### Run code & compare the records

In [56]:
Random.seed!(1234)
n, N = 10, 5
instances = [generate_random_symmetric_tsp(n) for _ in 1:N]

println("===== MTZ Results =====")
times_mtz, lengths_mtz = Float64[], Float64[]
for (i,C) in enumerate(instances)
    t,l = solve_mtz_tsp(C); push!(times_mtz,t); push!(lengths_mtz,l)
    println("Inst $i: Time=$(round(t; digits=4))s, Len=$l")
end

println("\n===== DFJ Results =====")
times_dfj, lengths_dfj = Float64[], Float64[]
for (i,C) in enumerate(instances)
    t,l = solve_dfj_tsp(C); push!(times_dfj,t); push!(lengths_dfj,l)
    println("Inst $i: Time=$(round(t; digits=4))s, Len=$l")
end

println("\n===== 2-opt Results =====")
times_2opt, lengths_2opt = Float64[], Float64[]
for (i,C) in enumerate(instances)
    t,l = solve_2opt_tsp(C); push!(times_2opt,t); push!(lengths_2opt,l)
    println("Inst $i: Time=$(round(t; digits=4))s, Len=$l")
end

println("\n===== Concorde Results =====")
times_c, lengths_c = Float64[], Float64[]
for (i,C) in enumerate(instances)
    t,l = solve_concorde_tsp(C); push!(times_c,t); push!(lengths_c,l)
    println("Inst $i: Time=$(round(t; digits=4))s, Len=$l")
end

println("\n===== LKH Results =====")
times_lkh, lengths_lkh = Float64[], Float64[]
for (i,C) in enumerate(instances)
    t,l = solve_lkh_tsp(C); push!(times_lkh,t); push!(lengths_lkh,l)
    println("Inst $i: Time=$(round(t; digits=4))s, Len=$l")
end

println("\n\nMTZ Avg: Time=",round(mean(times_mtz); digits=4),"s, Len=",round(mean(lengths_mtz); digits=2))
println("DFJ Avg: Time=",round(mean(times_dfj); digits=4),"s, Len=",round(mean(lengths_dfj); digits=2))
println("2-opt Avg: Time=",round(mean(times_2opt); digits=4),"s, Len=",round(mean(lengths_2opt); digits=2))
println("Concorde Avg: Time=",round(mean(times_c); digits=4),"s, Len=",round(mean(lengths_c); digits=2))
println("LKH Avg: Time=",round(mean(times_lkh); digits=4),"s, Len=",round(mean(lengths_lkh); digits=2))

===== MTZ Results =====
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2528975
Academic license 2528975 - for non-commercial use only - registered to ha___@kaist.ac.kr
Inst 1: Time=0.008s, Len=238.0
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2528975
Academic license 2528975 - for non-commercial use only - registered to ha___@kaist.ac.kr
Inst 2: Time=0.0084s, Len=215.0
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2528975
Academic license 2528975 - for non-commercial use only - registered to ha___@kaist.ac.kr
Inst 3: Time=0.0074s, Len=184.0
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2528975
Academic license 2528975 - for non-commercial use only - registered to ha___@kaist.ac.kr
Inst 4: Time=0.0069s, Len=177.0
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2528975
Academic license 2528975 - for non-com